### Test precedure for 4 european countries

### don t forget: UK and US might have several names in different languages!

In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine
import zipfile
import tarfile
import bz2
import sys

In [2]:
c_multilingu = pd.read_csv('./data/country_names_multilingual/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV')

FileNotFoundError: [Errno 2] File ./data/country_names_multilingual/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV does not exist: './data/country_names_multilingual/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV'

In [3]:
# lookup country codes - coordinates
co_from_cc = pd.read_csv('./data/location_ref/countrycodes_coord.csv')
country = co_from_cc[co_from_cc['country'] == 'AE']
(lat, long) = (country['latitude'].to_list()[0], country['longitude'].to_list()[0])
lat

FileNotFoundError: [Errno 2] File ./data/location_ref/countrycodes_coord.csv does not exist: './data/location_ref/countrycodes_coord.csv'

In [7]:
c_multilingu.head()

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_ALPHA3_CODE,COUNTRY_NUMERIC_CODE,COUNTRY_NAME
0,AF,AFRIKAANS,AD,AND,20,Andorra
1,AF,AFRIKAANS,AE,ARE,784,Verenigde Arabiese Emirate
2,AF,AFRIKAANS,AF,AFG,4,Afganistan
3,AF,AFRIKAANS,AG,ATG,28,Antigua en Barbuda
4,AF,AFRIKAANS,AI,AIA,660,Anguilla


In [8]:
## EU country codes
# https://abbreviations.yourdictionary.com/articles/list-of-europe-country-codes.html
# Austria  - AT - AUT - 43

# filter for Germany, Spain, France, Sweden
sel_4 = ['DE', 'ES', 'FR', 'SE' ]

# filter for European Countries
sel_european_un = ['AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR', 'DE',
                  'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL', 'PO', 'PT', 
                   'RO', 'SK', 'SI', 'ES', 'SE']
sel_c = sel_european_un

country_codes = c_multilingu[c_multilingu['COUNTRY_ALPHA2_CODE'].isin(sel_c)]
#country_codes

In [9]:
# GA - irish
# GL - galican
# LT - lithuanian
# LV - latvian
# lb - luxembourgish
# sk - slovak
# sl - slovenian
# nl - dutch, flemish
# ro - romanian

In [10]:
# filter for languages 
#https://www.loc.gov/standards/iso639-2/php/code_list.php
lang_4 = ['DE', 'FR', 'ES', 'SV']   
# EU + norway 
lang_european_un = ['DE', 'FR', 'HR', 'CS', 'DA', 'ET', 'FI', 
                    'EL',  'HU', 'GA', 'GL', 'IT', 'LT', 'LV', 'LB',
                    'SK', 'SL', 'NL', 'PL', 'PT', 'RO', 'SV', 'NN','ES', 'SV']

# assign current used languages list to "lang"
lang = lang_european_un

In [11]:
sel = c_multilingu[c_multilingu['LANG'].isin(lang)]
# # test
sel[sel['COUNTRY_NAME']=='Man']

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_ALPHA3_CODE,COUNTRY_NUMERIC_CODE,COUNTRY_NAME
13798,NN,NORWEGIAN NYNORSK,IM,IMN,833,Man


In [12]:
c_nam_list = sel['COUNTRY_NAME'].to_list()

# remove problem causing country names 
c_nam_list.remove('Man')


# get unique country names
c_nam_list_unique = list(set(c_nam_list))
len(c_nam_list), len(c_nam_list_unique)

(5726, 3364)

In [13]:
tweet_data = []
with open("./data/06/01/00/30.json") as f:
    for line in f:
        tweet_data.append(json.loads(line))

In [29]:
# check the structure of the jsons
tweet_data[0]

{'created_at': 'Mon Jun 01 06:30:00 +0000 2020',
 'id': 1267342617102123016,
 'id_str': '1267342617102123016',
 'text': 'RT @Aaleeetorres_: Corrimos como 2 cuadras y los perdimos a la yuta jajajaj',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1205152203742339079,
  'id_str': '1205152203742339079',
  'name': 'Niquitoo⚡',
  'screen_name': 'Nicooo35914277',
  'location': None,
  'url': None,
  'description': 'river plateee💓🤙🏽',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 272,
  'friends_count': 298,
  'listed_count': 0,
  'favourites_count': 5907,
  'statuses_count': 685,
  'created_at': 'Thu Dec 12 15:47:56 +0000 2019',
  'utc_offset': None,
  'time_zone': None,
  'geo_enabled': False,


In [15]:
# place:
# {
#     'id': '09c3311ea17328e3', 
#     'url': 'https://api.twitter.com/1.1/geo/id/09c3311ea17328e3.json', 
#     'place_type': 'city', 
#     'name': 'Haldwani-Kathgodam', 
#     'full_name': 'Haldwani-Kathgodam, India', 
#     'country_code': 'IN', 
#     'country': 'India',
#     'bounding_box': {
#         'type': 'Polygon', 'coordinates': [[[
#             79.125472, 28.982011], [79.125472, 29.414743], 
#             [79.98024, 29.414743], [79.98024, 28.982011]]]}, 'attributes': {}
# }

### Write to postgres database

In [16]:
# Declare postgres config
HOST = 'localhost'
USERNAME = 'postgres'
PORT = '5432'
DB = 'tweets_multilingu'
PASSWORD = 'postgres'

# Create a postgres connection and assign it to 'engine' variable
engine = create_engine(f'postgres://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}')

In [17]:
def get_to_country_coords(country_name_multilingual):
    # Get country coords
    return '79.125472, 28.982011'

def get_from_country_coords(country_code):
    """Function to retrieve country coordinates from ISO country codes"""
    country = co_from_cc[co_from_cc['country'] == country_code]
    (lat, long) = (country['latitude'].to_list()[0], country['longitude'].to_list()[0])
    # Get country coords
    return f'{lat}, {long}'

def get_coordinates(bounding_coords, city_name, user_location):
    if bounding_coords:
        return bounding_coords
    elif city_name:
        # Lookup city name coordinates
        return '79.125472, 28.982011'
    elif user_location:
        # Lookup user location
        return '79.125472, 28.982011'
    elif country_code:
        # Lookup country code
        return get_from_country_coords(country_code)

def extract_url(urls):
    found_url = ''
    url_given = urls
    if url_given != None:
        for url in url_given:
            found_url = url.get('url')
    return found_url

In [26]:
def load(data):
    """ Loads transformed tweeets into the Postgres database
    Parameters:
    ------------
    tweets: List of tweet dictionaries that were retrieved from the Twitter API """

    insert_query = """INSERT INTO tweets_test (
        tweet_id, 
        timestamp_ms,
        tweet, 
        url_in_tweet, 
        user_url, 
        user_location, 
        user_followers_count, 
        user_verified, 
        user_statuses_count,
        user_id,
        user_created_at,

        lang,  
        from_coords,
        from_country_coords,
        to_country_coords,
        tweet_country_code)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
    
    counter = 0 
    counter_loc = 0 
    counter_country_ment = 0 
    # loop through list of jsons 
    for j in data:
        # check for valid entries and if tweet json contains any location information 
        valid_entry = j.get('created_at')
        user_loc = j.get('user', {}).get('location')
        place_attr = j.get('place')
        user_url = j.get('user', {}).get('url')
        urls = j.get('entities', {}).get('urls')
        
        counter += 1 
        
        
        if valid_entry != None and len(urls) != 0 and user_url != None and user_loc != None or place_attr != None:
            print('entry with user loc or place attribute found')
            counter_loc += 1 

            
            # loop through multilingual country list 
            for c_name_multi in c_nam_list_unique:
                
                
                # check if one of the country names exist in tweet text
                if c_name_multi in j['text']:
                    counter_country_ment += 1
                    print(c_name_multi, 'in tweet found')

                    # get user dict
                    user = j.get('user')
                    # ger user_location if exits
                    user_location = user.get('location')
                            
                    # get place dict if exists and extract first coordinate of the bounding box
                    place = j.get('place')
                    country_code = ''
                    tweet_country_code = ''
                    bounding_coords = ''
                    city_name = ''
                    from_coords = ''
                    from_country_coords = ''

                    if place != None:
                        country_code = place.get('country_code')
                        tweet_country_code = place.get('country_code')
                        
                        bounding_coords = ','.join(list(map(lambda n: str(n), place.get('bounding_box').get('coordinates')[0][0])))
                        
                        city_name = place.get('full_name')
                        from_coords = get_coordinates(bounding_coords, city_name, user_location)
                        from_country_coords = get_from_country_coords(country_code)
                        
                    to_country_coords = get_to_country_coords(c_name_multi)
                    
                    # get url mentionend in tweet from entities dict
                    found_url = extract_url(j.get('entities').get('urls'))
                    

                    # insert values into postgres DB                             
                    engine.execute(insert_query, (
                        j['id'], #id
                        j['timestamp_ms'], #timestamp_ms
                        j['text'], #tweet
                        found_url, #url_in_tweet
                        user['url'], #user_url
                        user_location, #user_location
                        user['followers_count'], #user_followers_count
                        user['verified'], #user_verified
                        user['statuses_count'], #user_statuses_count
                        user['id'], #user_id
                        user['created_at'], #user_created_at
                        
                        j['lang'], #lang
                        from_coords, #from_coords
                        from_country_coords, #from_country_coords
                        to_country_coords, #to_country_coords
                        tweet_country_code #tweet_country_code
                        )
         
                    )
    
    
    print('scanned jsons :', counter, 
          '\n with some location information and user url given :', counter_loc,
          '\n with some location information, user url and country name mentioned :',counter_country_ment)
    
        
### MORE filters:
# follwers count above certain threshold! - maybe filter that in postgres!?!


# how to make sure that country mentioned is the same as from location country!?!
# also in postgres? if 'countryname in local language == contains countryname in tweet'

In [27]:
load(tweet_data)

entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with user loc or place attribute found
entry with

In [ ]:
#### write os walk through all the files for the whole month